# Laboratorio 8

## Task 1 - Preguntas

1. **Investigar el algoritmo AC-3 y su relación con el algoritmo de backtracking search**<br>
AC-3 o Arc Consistency 3 es un método que se utiliza en problemas de satisfacción de restricciones para reducir el dominio de las variables antes de realizar una búsqueda. Su relación con el Backtracking Search es debido a que el AC-3 mejora la eficiencia de la búsqueda eliminando valores inconsistentes antes de que el backtracking los explore. 

2. **Defina en sus propias palabras el término “Arc Consistency”**<br>
Es una propiedad en los problemas de satisfacción de restricciones que dice que para cada par de variables con restricciones, cada valor posible en una variable tiene al menos un valor compatible en la otra. 

## Task 2 - Algoritmos CSP

In [9]:
import time
import random
from collections import defaultdict
import numpy as np

# Variables
exams = ['Matemática', 'Física', 'Química', 'Historia', 'Inglés', 'Biología', 'Computación']
days = ['Lunes', 'Martes', 'Miércoles']

# Dominio inicial
domain = {exam: days[:] for exam in exams}

# Cursos por estudiante
students = {
    'Ana': ['Matemática', 'Física', 'Inglés'],
    'Luis': ['Física', 'Historia', 'Computación'],
    'Sofía': ['Matemática', 'Química', 'Biología'],
    'Carlos': ['Historia', 'Biología', 'Inglés']
}

# Relación: cursos que no deben ser el mismo día si los comparten estudiantes
def conflict(c1, c2):
    for cursos in students.values():
        if c1 in cursos and c2 in cursos:
            return True
    return False

# Generar pares de cursos conflictivos
conflict_pairs = set()
for i in range(len(exams)):
    for j in range(i + 1, len(exams)):
        if conflict(exams[i], exams[j]):
            conflict_pairs.add((exams[i], exams[j]))


### Backtracking Search

In [10]:
def is_valid(assignment, var, value):
    for other_var, other_val in assignment.items():
        if value == other_val and ((var, other_var) in conflict_pairs or (other_var, var) in conflict_pairs):
            return False
    return True


def backtracking(assignment):
    if len(assignment) == len(exams):
        return assignment

    unassigned = [v for v in exams if v not in assignment]
    var = unassigned[0]
    for value in days:
        if is_valid(assignment, var, value):
            assignment[var] = value
            result = backtracking(assignment)
            if result:
                return result
            del assignment[var]
    return None

# Ejecutar y medir tiempo
start_time = time.perf_counter()
backtracking_result = backtracking({})
end_time = time.perf_counter()
backtracking_time = end_time - start_time

print("Backtracking Result:", backtracking_result)
print("Execution Time:", end_time - start_time, "seconds")


Backtracking Result: {'Matemática': 'Lunes', 'Física': 'Martes', 'Química': 'Miércoles', 'Historia': 'Lunes', 'Inglés': 'Miércoles', 'Biología': 'Martes', 'Computación': 'Miércoles'}
Execution Time: 5.8999983593821526e-05 seconds


### Beam Search

In [11]:
def evaluate(state):
    """Evalúa cuántos conflictos tiene el estado actual"""
    conflicts = 0
    for (a, b) in conflict_pairs:
        if a in state and b in state and state[a] == state[b]:
            conflicts += 1
    return conflicts

def beam_search(k=3):
    beam = [{}]  # Lista de posibles soluciones parciales
    while True:
        new_beam = []
        for partial in beam:
            unassigned = [v for v in exams if v not in partial]
            if not unassigned:
                new_beam.append(partial)
                continue
            var = unassigned[0]
            for day in days:
                new_assignment = partial.copy()
                new_assignment[var] = day
                new_beam.append(new_assignment)

        # Evaluar todos y quedarse con los k mejores
        new_beam = sorted(new_beam, key=lambda x: evaluate(x))[:k]
        # Si todos están completos y sin conflictos, retornar
        for candidate in new_beam:
            if len(candidate) == len(exams) and evaluate(candidate) == 0:
                return candidate
        beam = new_beam

start_time = time.perf_counter()
beam_result = beam_search(k=3)
end_time = time.perf_counter()

print("Beam Search Result:", beam_result)
print("Execution Time:", end_time - start_time, "seconds")

Beam Search Result: {'Matemática': 'Lunes', 'Física': 'Martes', 'Química': 'Miércoles', 'Historia': 'Lunes', 'Inglés': 'Miércoles', 'Biología': 'Martes', 'Computación': 'Miércoles'}
Execution Time: 0.00012169999536126852 seconds


### Local Search - Hill Climbing

In [12]:
def random_assignment():
    return {exam: random.choice(days) for exam in exams}

def count_conflicts(state):
    conflicts = 0
    for (a, b) in conflict_pairs:
        if state[a] == state[b]:
            conflicts += 1
    return conflicts

def get_neighbors(state):
    neighbors = []
    for exam in exams:
        for day in days:
            if state[exam] != day:
                neighbor = state.copy()
                neighbor[exam] = day
                neighbors.append(neighbor)
    return neighbors

def hill_climbing(max_iterations=1000):
    current = random_assignment()
    current_conflicts = count_conflicts(current)

    for _ in range(max_iterations):
        neighbors = get_neighbors(current)
        best_neighbor = min(neighbors, key=count_conflicts)
        best_conflicts = count_conflicts(best_neighbor)

        if best_conflicts >= current_conflicts:
            break  # No mejor vecino, llegamos a un óptimo local
        current = best_neighbor
        current_conflicts = best_conflicts

        if current_conflicts == 0:
            break

    return current if current_conflicts == 0 else None

In [13]:
# Ejecutar y medir tiempo
# Reintenta hasta que encuentre solución
solution = None
attempts = 0
max_attempts = 50

start_time = time.perf_counter()
while not solution and attempts < max_attempts:
    solution = hill_climbing()
    attempts += 1
end_time = time.perf_counter()

print(f"Local Search Result: {solution}")
print(f"Execution Time: {end_time - start_time} seconds")
print(f"Attempts: {attempts}")

Local Search Result: {'Matemática': 'Lunes', 'Física': 'Martes', 'Química': 'Miércoles', 'Historia': 'Lunes', 'Inglés': 'Miércoles', 'Biología': 'Martes', 'Computación': 'Miércoles'}
Execution Time: 0.00030129996594041586 seconds
Attempts: 3


## Conclusiones

En general todos los algoritmos funcionaron rápido y dando una solución real. Esto debido a que el problema no era muy grande, al contar solo con 7 días y cuatro estudiantes. En este caso vemos que el Backtracking fue el más veloz y esto puede deberse a que no depende de parametros heurísticos. Los otros dos fueron más lento pero no lo suficiente para ser significativos al momento de escoger una herramienta para resolver un problema con estas características (en especial la cantidad de datos).